In [ ]:
# https://ratsgo.github.io/speechbook/docs/fe/mfcc (MFCC 만들기)
# https://youtu.be/jYgMMy8KAc0 (Mel-frequency cepstrum(MFCC))
import scipy.io.wavfile
sample_rate, signal = scipy.io.wavfile.read('example.wav')

In [ ]:
# 16KHz (초당 16000번 Sampling)
print(f"sampling Rate : {sample_rate}")
# 음악 데이터
print(f"signal Array : {signal}")
# 데이터가 16비트로 양자화 (아날로그 신호가 -32768 ~ 32767 사이의 값으로 Mapping)
print(f"signal Dtype : {signal.dtype}")
# 데이터 길이
print(f"signal Length : {len(signal)}")
# Sampling Data를 초당 Sampling rate로 나누면 음원이 몇 초인지 알 수 있다.
print(f"music length : {len(signal) / sample_rate} sec")

sampling Rate : 16000
signal Array : [36 37 60 ...  7  9  8]
signal Dtype : int16
signal Length : 183280
music length : 11.455 sec


In [ ]:
# 편의를 위해 처음 3.5초 자름
signal = signal[0:int(3.5 * sample_rate)]
print(f"signal Length : {len(signal)}")

signal Length : 56000


In [ ]:
# Preemphasis
# 고주파 성분의 에너지를 올려주는 전처리 과정

# 상대적으로 에너지가 적은 고주파 성분을 강화함으로써 원시 음성 신호가 전체 주파수 영역대에서 비교적 고른 에너지 분포를 갖도록 함.
# 푸리에 변환시 발생할 수 있는 numerical problem 예방.
# Signal-to-Noise Ratio(SNR) 개선.

import numpy as np
pre_emphasis = 0.97
emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

In [ ]:
# Framing
# 원시 음성 신호를 잘게 쪼갠다. 단, 잘게 자르되 (예시 : 25ms) 일정 구간 (fame stride = 10ms) 씩 겹치게 한다.
# 오디오 신호를 짧게 자르면(20~40ms), 그 오디오 신호 내에서의 발음을 stationary (변화하지 않는다) 고 가정할 수 있다.
# 만약 사람이 이야기를 한다고 하더라도, 20ms을 잘라서 본다면 그 사람은 같은 음소를 발음중이라고 간주할 수 있다.

# 또한, FFT는 Infinite Cycle을 가정하므로, 짧은 단위로 잘라 준다.


# 25ms
frame_size = 0.025
# 10ms
frame_stride = 0.01
# frame length / step : 시간 * sampling rate
frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate

# 원 signal 길이 => signal_length
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))

# frame의 개수 : -(-signal_length//Frame_step)
num_frames = int(np.ceil(float(signal_length - frame_length) / frame_step))

# 넘치는 부분은 zero padding 해 준다.
pad_signal_length = num_frames * frame_step + frame_length
z = np.zeros((pad_signal_length - signal_length))
#pad_signal : zero padding한 데이터
pad_signal = np.append(emphasized_signal, z)

# indices : 원본 signal에서 index 기준으로 값 찾기 위한 임시변수
indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(np.int32, copy=False)]

print(f"signal_length : {signal_length}")
print(f"frame_length : {frame_length}")
print(f"frame_step : {frame_step}")
print(f"num_frames : {num_frames}")


print(f"pad Signal : {pad_signal}")
print(f"pad Signal Length : {len(pad_signal)}")


print("indices : (원본 signal에서 index 기준으로 값 찾기 위한 임시변수)")
print(indices)


print("frames:")
print(frames)

print(f"frames.shape : {frames.shape}")

signal_length : 56000
frame_length : 400
frame_step : 160
num_frames : 348
pad Signal : [36.    2.08 24.11 ...  0.    0.    0.  ]
pad Signal Length : 56080
indices : (원본 signal에서 index 기준으로 값 찾기 위한 임시변수)
[[    0     1     2 ...   397   398   399]
 [  160   161   162 ...   557   558   559]
 [  320   321   322 ...   717   718   719]
 ...
 [55200 55201 55202 ... 55597 55598 55599]
 [55360 55361 55362 ... 55757 55758 55759]
 [55520 55521 55522 ... 55917 55918 55919]]
frames:
[[  36.      2.08   24.11 ...    4.56    3.74    2.89]
 [  16.43  -32.15  -47.2  ...  -13.06  -16.45    2.07]
 [  -9.     -9.27   11.46 ...   -5.09   -7.24   -2.45]
 ...
 [ 315.7   130.65  211.81 ... -121.15  -17.69 -195.02]
 [ 283.62 1098.42  815.34 ...   20.53  136.92  150.79]
 [ -59.03 -212.81 -289.18 ... -157.35  -81.12   24.54]]
frames.shape : (348, 400)


In [ ]:
# Windowing
# 각 Frame의 경계를 Smoothing. Frame 단위로 네모나게 자르면 연속적 데이터가 아니게 된다.
# 연속적 형태로 바꿔주기 위해 Hamming window 사용한다. (Hann Window도 있다)
# 즉, 양 끝에서 Smoothing해준다.

# Hamming Window 적용
frames *= np.array([0.54 - 0.46 * np.cos((2 * np.pi * n) / (frame_length - 1)) for n in range(frame_length)])

In [ ]:
# Fourer Transform

# 고속 푸리에 변환 사용

# NFFT : Frequency Domain 변환 시 몇개의 구간(Bin) 으로 분석할 지 나타내는 인자
# 보통 512 혹은 256 사용한다고 알려져 있다..
NFFT = 512

# 푸리에 변환 결과는 켤레대칭 (실수부 같고 허수부만 반대)
# fft는 켤레대칭 전부 계산, rfft는 켤레 대칭부 생략하여 연산 속도가 빠르다.
dft_frames = np.fft.rfft(frames, NFFT)

# 어떤 함수를 쓰던 이후의 Mel Spectrum, MFCC 계산 결과등이 달라지지 않으므로 rfcc를 쓰는 것이 연산량 줄이는데 좋다.
# fft 결과의 shape -> num_frames * NFFT (이 경우, Filter Bank 차원수는 NFFT)
# rfft 결과의 shape -> num_frames * NFFT / 2 + 1 (이 경우, Filter Bank 차원수는 NFFT / 2 + 1)

print(dft_frames)

[[  18.58681572  +0.j          -14.01988178 -84.91698726j
   -24.70392533+106.88963159j ...  -26.66158624  -5.85474324j
     0.92680879 +28.72849855j   32.82338322  +0.j        ]
 [-142.51526149  +0.j           85.6674828 +108.25845827j
    13.38303476-108.51765447j ...  -10.58513364  +4.31215777j
     7.05534013  -2.14342983j   -3.03115655  +0.j        ]
 [ -19.2843489   +0.j          -15.14198098 -16.21735682j
   -40.12895986 +59.02120051j ...   -4.25775098 -14.59761671j
   -10.25228518  +7.21787503j    8.21971695  +0.j        ]
 ...
 [ -20.99872977  +0.j         -173.80587746 -75.85843408j
  -120.10047358+121.33988075j ...  -36.66157943-135.12987296j
  -150.42479757 -89.13659856j  214.56477173  +0.j        ]
 [ 223.33492956  +0.j          268.55310205 +78.36772313j
   177.86153856 +13.40968462j ... -471.85194623-511.29061637j
    45.11248225+373.13984952j  166.92458692  +0.j        ]
 [ -70.92601653  +0.j          145.29755513 +30.95048802j
     9.87980069 -51.04003282j ...  259.850

In [ ]:
# Magnitude
# 이산 푸리에 변환의 결과 : a + bj (j는 허수)
# 해당 주파수 성분의 진폭(magnitude) : sqrt(a^2 + b^2)
# 해당 주파수 성분의 위상 (tan^-1(b/a))

# 이때, MFCC에서 필요한 것은 진폭(magnitude), 따라서 위상은 제거한다.

# np.absolute 함수 : 복소수 입력(a + bj) 들어오면 sqrt(a^2 + b^2) 리턴한다.
mag_frames = np.absolute(dft_frames)
print(mag_frames)

[[ 18.58681572  86.06655454 109.70723435 ...  27.29685328  28.74344453
   32.82338322]
 [142.51526149 138.05365405 109.33977754 ...  11.4297751    7.37374503
    3.03115655]
 [ 19.2843489   22.18743451  71.37111131 ...  15.20588232  12.53822441
    8.21971695]
 ...
 [ 20.99872977 189.6390916  170.72636122 ... 140.01483483 174.85123085
  214.56477173]
 [223.33492956 279.75394305 178.36632681 ... 695.74589726 375.85699854
  166.92458692]
 [ 70.92601653 148.55743749  51.98745437 ... 385.55333259 183.12959323
   11.68459131]]


In [ ]:
# Power Spectrum

# 이산 푸리에 변환 값을 바탕으로 Power 구한다. 진폭(magnitude)의 제곱을 N으로 나눠준 값.
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))

print(len(pow_frames[0]))
print(pow_frames)


257
[[6.00906490e+00 5.63201702e+00 4.55372311e+00 ... 2.57019644e-01
  5.39075402e-02 3.11548603e-04]
 [1.55191646e+01 1.38241442e+01 9.69142992e+00 ... 4.41034557e-02
  2.29938310e-02 1.35983781e-02]
 [1.40697212e+00 2.07923377e+00 3.84174885e+00 ... 2.68078815e-01
  2.41497669e-01 2.25262340e-01]
 ...
 [7.55643460e+00 1.32400333e+02 6.42979480e+02 ... 5.91145335e+01
  5.32001479e+01 5.06704454e+01]
 [3.33436158e+01 1.53294596e+02 6.62955386e+02 ... 5.14080902e+02
  3.78806639e+02 3.20121373e+02]
 [7.97582473e+01 1.75171834e+02 6.30672091e+02 ... 4.93477244e+01
  4.78739289e+01 4.70020223e+01]]


In [ ]:
# Filter Banks

# 사람의 소리 인식은 1000Hz 이하의 저주파(low frequency) 영역대가 고주파(high frequency) 대비 민감하다고 알려져 있음
# 따라서, 주파수별 에너지 정보가 있는 데이터(pow_frames)에서 저주파수 영역대를 고주파수 영역대보다 상대적으로 세밀하게 볼 필요가 있음.
# 이를 위해 Filter Banks를 사용 

# Filter Bank는 Mel Scale 필터를 사용. 따라서 기존 주파수(f, 단위 : Hz) -> 멜(m, 단위 : mel)로 변환 필요.
# 각 단위의 변환 공식은 다음과 같다.
# m = 2595 log10(1 + f/700)
# f = 700(10^m/2595 -1)

# Mel-Filter의 IDEA : 1000Hz까지는 Linear하게, 이후는 Log Scale로 변환하자!
# Mel Scale Filter는 수식 참고하자. (복잡하다)

# nfilt : 필터의 개수, 보통 26~40개정도 사용
nfilt = 40
low_freq_mel = 0
high_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = np.floor((NFFT + 1) * hz_points / sample_rate)

fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right
    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

# 필터의 개수 : nfilt와 같다. 26~40개로 Parmeter로 사용 가능
print(f"number of filters : {len(fbank)}")

# 필터의 길이 : NFFT / 2 + 1로 고정 (위 FFT 변환 참고)
print(f"filter Length : {len(fbank[0])}")

# 뒤로 갈수록, 고주파수에 걸쳐 넓게 분포하고 있는 필터임을 확인할 수 있다.

# Filter 0은 저주파수를 세밀하게 살피는 필터
print("--filter 0--")
print(fbank[0])

print("--filter 3--")
print(fbank[3])

# Filter 39는 고주파수 영역대를 넓게 보는 필터
print("--filter 39--")
print(fbank[39])

number of filters : 40
filter Length : 257
--filter 0--
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--filter 3--
[0.  0.  0.  0.  0.  0.5 1.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0. 

In [ ]:
# Filter Bank 기법 적용. Dot Product 한다.
# Filter[0] 과 DOT PRODUCT 하면 저주파의 
filter_banks = np.dot(pow_frames, fbank.T)
filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability

print(filter_banks.shape)
print(filter_banks)

(348, 40)
[[5.63201702e+00 6.09295405e+00 3.92456890e+00 ... 3.35293300e+01
  1.49382397e+01 5.82492107e+01]
 [1.38241442e+01 1.22772856e+01 4.66339420e+00 ... 1.36506497e+01
  1.23567736e+01 1.55865797e+01]
 [2.07923377e+00 6.83181799e+00 1.44543176e+01 ... 1.48822731e+01
  1.67528111e+01 2.09883201e+01]
 ...
 [1.32400333e+02 1.55905191e+03 7.28789248e+03 ... 4.64667970e+03
  2.66257237e+02 9.04374585e+02]
 [1.53294596e+02 1.60334518e+03 7.48150355e+03 ... 2.25589055e+03
  1.84045289e+03 6.59654198e+03]
 [1.75171834e+02 1.52941147e+03 7.33911598e+03 ... 1.46680252e+04
  1.45087902e+03 2.16246224e+03]]


In [ ]:
# Log-Mel Spectrum

# 사람의 귀는 진폭(magnitude)에도 log 스케일에 가깝게 반응
# 즉, 두배 큰 소리라고 인식하려면 실제로는 에너지가 100배 커야 한다.
# 따라서, Mel 스펙트럼에 로그 변환을 수행

filter_banks = 20 * np.log10(filter_banks)  # dB

# 모든 변환 수행 후 차원수 : num_frames * nfilt
print(filter_banks.shape)
print(filter_banks)

(348, 40)
[[15.01327917 15.69655805 11.87583916 ... 30.50849749 23.4859885
  35.30580089]
 [22.81276513 21.78204715 13.37404257 ... 22.70306644 21.83810183
  23.8550165 ]
 [ 6.35806641 16.69072575 23.19995185 ... 23.45338539 24.48175385
  26.4395536 ]
 ...
 [42.43778156 63.85721153 77.25203914 ... 73.34285475 48.50602841
  59.12696698]
 [43.71053693 64.10054061 77.47977773 ... 67.06636052 65.2984941
  76.38632662]
 [44.86928552 63.69048688 77.31287502 ... 83.32743297 63.23262402
  66.69897066]]


In [125]:
# MFCCs

# Mel Spectrum은 Feature 변수 내 상관관계가 존재.
# Mel Scale Filter 볼 때, 특정 주파수 영역대 에너지를 모아서 본다.
# 즉, Hz 기준의 특정 주파수 영역대의 에너지 정보가 Mel Spectrum, Log Mel Spectrum에 영향을 주는 구조
# 이런 경우, 변수 간 독립(independence) 가정하는 Model 등에는 독이 될 수 있음.

# 따라서, Log Mel Spectrum에 역 푸리에 변환(Inverse Fourier Transform) 수행해 변수 간 상관관계 해소한 Feature을 MFCCs (Mel-frequency Cepstral Coefficients) 라 함.
# 역 푸리에 변환을 실시하면 Freq Domain에서 다른 도메인으로 바뀌어 (상대적으로) 변수간 상관관계 문제가 해소됨.

# MFCCs 만들땐 역 이산 코사인 변환(Inverse Discrete Cosine Transform) 실행
# 변환 후 Shape : (num_frames * nfilt) 이지만, 여기서 2-13번째 열 벡터만 뽑아 Feaure로 주로 사용
# num_ceps(뽑을 벡터) 하이퍼파라미터로 조절 가능하다. 전통적으론 처음 12~13 Frame 뽑는 경우가 잦다. (정보량이 가장 많다고 알려져 있음)

# MFCCs 만들 때 버려지는 정보가 너무 많다. (열 개수 40 -> 12)
# 따라서, 최신 모델들에서는 MFCCs보다는 Mel Spectrum, Log Mel Spectrum 등 사용하는 경우가 많다.

from scipy.fftpack import dct
num_ceps = 12
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13

print(mfcc.shape)
print(mfcc)

(348, 12)
[[-65.87064078 -63.92533935  13.43154858 ...   9.38411191  12.72278725
   -5.94707833]
 [-45.88096119 -52.36350898  16.6459209  ...   8.43356197   6.21816814
    9.82593622]
 [-36.59785685 -55.74815248  13.57672542 ...  20.12950351   5.68368371
  -14.24620822]
 ...
 [  6.30751454 -25.97384046  16.99143892 ...  -0.17818009  -3.19056716
  -19.80233455]
 [ -0.44408993 -21.38180357   9.82475558 ...   8.7416867   -4.18883276
  -12.4263686 ]
 [  3.45746883 -28.04399993  10.35275558 ...  -4.83341103   7.8103839
  -19.69318886]]


In [ ]:
# Post Processing

# 음성 인식 시스템 성능을 높이기 위해 후처리 하기도한다.

# LIFT (MFCCs에 적용)
(nframes, ncoeff) = mfcc.shape
cep_lifter = 22
n = np.arange(ncoeff)
lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
mfcc *= lift

# Mean Normalization (Mel Spectrum, Log Mel Spectrum에 적용)
filter_banks -= (np.mean(filter_banks, axis=0) + 1e-8)